In [1]:
# Scan all source files and build full scan file
import datetime
import pandas as pd
import numpy as np
import os
from collections import defaultdict
import dask.dataframe as dd
import sys

In [19]:
s3_monitor_dir = '/Users/brianm/Dropbox/cc_AuroraInsight/data/s3/s3_monitor2/s3_monitor'
s3_results_dir = '/Users/brianm/Dropbox/cc_AuroraInsight/data/s3'

s3_results_file = 's3_scan7.csv'
s3_results_file_path = os.path.join(s3_results_dir, s3_results_file)

In [21]:
file_list = os.listdir(s3_monitor_dir)


In [22]:
len(file_list)

1746849

In [23]:
file_list = [name for name in file_list 
             if '.swp' not in name and 
             os.path.isfile(os.path.join(s3_monitor_dir, name))]


In [24]:
len(file_list)

1746849

In [25]:
s3_monitor_lists = []




In [26]:
columns = ['id', 'bucket', 'time', 'time_ms?','accessw_ip', 
           'arn', 'full_action', 'key', 'basic_action', 'key_extended', 
           'http', 'response', 'size', 'software_method',
           'token', 'sig_version',  'encoding', 'header', 's3_location',
           'aws_version'
          ]
needed_columns =  ['bucket', 'time', 'accessw_ip', 
          'full_action', 'key', 'basic_action', 'size', 
            'software_method'
                  ]



In [27]:
num_columns = 27

In [28]:
columns = ['col_{}'.format(x) for x in range(num_columns) ]

In [29]:
drop_columns = [
    'col_0',  
    'col_6',
    'col_11',
    'col_14',
    'col_15',
    'col_16',
    'col_19', 'col_20', 'col_21', 'col_22', 'col_23', 'col_24', ]

In [30]:
new_columns = [
    's3_bucket',
    'datetime',
    'ms',
    'source_ip',
    'arn',
    'operation',
    's3_key',
    'op_key',
    'http_resp',
    'part_size',
    'size',
    'sw_used',
    'unknown_1',
    'source_file',
    'source_dir'    
]

In [31]:
save_df = None

In [32]:
def process_df(this_df):
    global save_df
    this_df['col_2'] = this_df['col_2'].map(lambda x: x.replace('[',''))
    this_df['col_2'] = this_df['col_2'].map(lambda x: datetime.datetime.strptime(x,'%d/%b/%Y:%H:%M:%S'))
    this_df['col_3'] = this_df['col_3'].map(lambda x: x.replace(']',''))
    this_df = this_df.drop(drop_columns, axis=1)
    this_df.columns = new_columns
    save_df = this_df
    #print(this_df)
    this_df['s3_key'] = this_df['s3_key'].apply(str)
    this_df['prefix'] = this_df['s3_key'].map(lambda x: "/".join(x.split('/')[:-1]))
    this_df['file'] = this_df['s3_key'].map(lambda x: x.split('/')[-1])
    save_df = this_df
    if '-' in list(this_df['size'].values):
        this_df['size'] = this_df['size'].map(lambda x: -1 if '-' in str(x) else x)
    if '-' in list(this_df['part_size'].values):
        this_df['part_size'] = this_df['part_size'].map(lambda x: -1 if '-' in str(x) else x)
      
    return this_df

In [33]:
length_counts = defaultdict(int)
first_time = True
for file_name in file_list:
    df_content = pd.read_csv(
        os.path.join(s3_monitor_dir, file_name),
        delimiter=' ',
        quotechar='"',
        header=None )
    #print(file_name)
    df_content.columns = columns[:25]
    length_counts[len(df_content.columns)] += 1
    df_content['source_file'] = file_name
    df_content['source_dir'] = "."
    df_content = process_df(df_content.copy())
    num_columns = len(df_content.columns)
    if first_time:
        first_time = False
        df_content.to_csv(s3_results_file_path, mode='w', header=True, index=True)               
    df_content.to_csv(s3_results_file_path, mode='a', header=False, index=True)


    



  
    
    

In [34]:
save_df

,s3_bucket,datetime,ms,source_ip,arn,operation,s3_key,op_key,http_resp,part_size,size,sw_used,unknown_1,source_file,source_dir,prefix,file
0,aii-data,2020-10-12 15:05:09,+0000,65.140.70.186,arn:aws:iam::874847117436:user/upload_app,REST.PUT.PART,v1/dev/terrestrial/L0/UK/20201008/2020-09-27T0...,PUT /v1/dev/terrestrial/L0/UK/20201008/2020-09...,200,-1,16777216,aws-cli/1.18.69 Python/3.6.9 Linux/5.4.0-42-ge...,-,2020-10-12-16-41-41-491EC75508E6F450,.,v1/dev/terrestrial/L0/UK/20201008/2020-09-27T0...,iq_out_RX2_318EE20_0_1601219456.sigmf-data
1,aii-data,2020-10-12 15:06:10,+0000,65.140.70.186,arn:aws:iam::874847117436:user/upload_app,REST.POST.UPLOAD,v1/dev/terrestrial/L0/UK/20201008/2020-09-27T0...,POST /v1/dev/terrestrial/L0/UK/20201008/2020-0...,200,552,218103808,aws-cli/1.18.69 Python/3.6.9 Linux/5.4.0-42-ge...,-,2020-10-12-16-41-41-491EC75508E6F450,.,v1/dev/terrestrial/L0/UK/20201008/2020-09-27T0...,iq_out_RX2_318EE20_0_1601219456.sigmf-data
2,aii-data,2020-10-12 15:06:11,+0000,65.140.70.186,arn:aws:iam::874847117436:user/upload_app,REST.PUT.PART,v1/dev/terrestrial/L0/UK/20201008/2020-09-27T0...,PUT /v1/dev/terrestrial/L0/UK/20201008/2020-09...,200,-1,16777216,aws-cli/1.18.69 Python/3.6.9 Linux/5.4.0-42-ge...,-,2020-10-12-16-41-41-491EC75508E6F450,.,v1/dev/terrestrial/L0/UK/20201008/2020-09-27T0...,iq_out_RX2_318EE20_0_1601221518.sigmf-data
3,aii-data,2020-10-12 15:07:37,+0000,65.140.70.186,arn:aws:iam::874847117436:user/upload_app,REST.PUT.PART,v1/dev/terrestrial/L0/UK/20201008/2020-09-27T0...,PUT /v1/dev/terrestrial/L0/UK/20201008/2020-09...,200,-1,16777216,aws-cli/1.18.69 Python/3.6.9 Linux/5.4.0-42-ge...,-,2020-10-12-16-41-41-491EC75508E6F450,.,v1/dev/terrestrial/L0/UK/20201008/2020-09-27T0...,iq_out_RX2_318EE20_0_1601221747.sigmf-data
4,aii-data,2020-10-12 15:26:02,+0000,64.124.142.126,arn:aws:iam::874847117436:user/upload_app,REST.PUT.PART,v1/dev/terrestrial/L0/UK/20201008/2020-09-30T0...,PUT /v1/dev/terrestrial/L0/UK/20201008/2020-09...,400,416,16777216,aws-cli/1.18.69 Python/3.6.9 Linux/5.4.0-42-ge...,-,2020-10-12-16-41-41-491EC75508E6F450,.,v1/dev/terrestrial/L0/UK/20201008/2020-09-30T0...,iq_out_RX2_318EE20_1_1601481541.sigmf-data
5,aii-data,2020-10-12 15:26:02,+0000,64.124.142.126,arn:aws:iam::874847117436:user/upload_app,REST.PUT.PART,v1/dev/terrestrial/L0/UK/20201008/2020-09-30T0...,PUT /v1/dev/terrestrial/L0/UK/20201008/2020-09...,200,-1,16777216,aws-cli/1.18.69 Python/3.6.9 Linux/5.4.0-42-ge...,-,2020-10-12-16-41-41-491EC75508E6F450,.,v1/dev/terrestrial/L0/UK/20201008/2020-09-30T0...,iq_out_RX2_318EE20_1_1601481656.sigmf-data
6,aii-data,2020-10-12 15:26:02,+0000,64.124.142.126,arn:aws:iam::874847117436:user/upload_app,REST.PUT.PART,v1/dev/terrestrial/L0/UK/20201008/2020-09-15T0...,PUT /v1/dev/terrestrial/L0/UK/20201008/2020-09...,400,416,16777216,aws-cli/1.18.69 Python/3.6.9 Linux/5.4.0-42-ge...,-,2020-10-12-16-41-41-491EC75508E6F450,.,v1/dev/terrestrial/L0/UK/20201008/2020-09-15T0...,iq_out_RX2_318EE20_1_1600177413.sigmf-data
7,aii-data,2020-10-12 15:26:03,+0000,64.124.142.126,arn:aws:iam::874847117436:user/upload_app,REST.PUT.PART,v1/dev/terrestrial/L0/UK/20201008/2020-09-30T0...,PUT /v1/dev/terrestrial/L0/UK/20201008/2020-09...,200,-1,16777216,aws-cli/1.18.69 Python/3.6.9 Linux/5.4.0-42-ge...,-,2020-10-12-16-41-41-491EC75508E6F450,.,v1/dev/terrestrial/L0/UK/20201008/2020-09-30T0...,iq_out_RX2_318EE20_1_1601481753.sigmf-data
8,aii-data,2020-10-12 15:26:03,+0000,64.124.142.126,arn:aws:iam::874847117436:user/upload_app,REST.PUT.PART,v1/dev/terrestrial/L0/UK/20201008/2020-09-30T0...,PUT /v1/dev/terrestrial/L0/UK/20201008/2020-09...,400,416,16777216,aws-cli/1.18.69 Python/3.6.9 Linux/5.4.0-42-ge...,-,2020-10-12-16-41-41-491EC75508E6F450,.,v1/dev/terrestrial/L0/UK/20201008/2020-09-30T0...,iq_out_RX2_318EE20_1_1601481541.sigmf-data
9,aii-data,2020-10-12 15:26:04,+0000,64.124.142.126,arn:aws:iam::874847117436:user/upload_app,REST.PUT.PART,v1/dev/terrestrial/L0/UK/20201008/2020-09-27T0...,PUT /v1/dev/terrestrial/L0/